# test file : ID, Lidar file, Image file, Calib

## load lib

In [2]:
import numpy as np
import pandas as pd
import os
import gc


import json
import math 
import sys
from datetime import datetime
import time
from typing import Tuple,List

import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import sklearn.metrics
from PIL import Image

from matplotlib.axes import Axes
from matplotlib import animation, rc
import plotly.graph_objs as go
import plotly.tools as ts
from plotly.offline import plot, init_notebook_mode
import plotly.figure_factory as ft
init_notebook_mode(connected=True)
from pyquaternion import Quaternion
import seaborn as sns
from tqdm import tqdm 
import warnings

from lyft_dataset_sdk.utils.map_mask import MapMask
from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.geometry_utils import view_points, box_in_image, BoxVisibility
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix
from pathlib import Path

from lyft_dataset_sdk.lyftdataset import LyftDataset,LyftDatasetExplorer
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix
import time
from lyft_dataset_sdk.utils.map_mask import MapMask

import struct
from abc import ABC, abstractmethod
from functools import reduce
from typing import Tuple, List, Dict
import copy
from shutil import copyfile

from PIL import Image

## link folders

In [34]:
!ln -s ../../3d-object-detection-for-autonomous-vehicles/test_images images
!ln -s ../../3d-object-detection-for-autonomous-vehicles/test_maps maps
!ln -s ../../3d-object-detection-for-autonomous-vehicles/test_lidar lidar

In [9]:
plt.rcParams['figure.figsize']=[16,10]
plt.rcParams['font.size']=14
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 99
sns.set_palette(sns.color_palette('tab20', 20))
#Path to the dataset
#Path= '../input/3d-object-detection-for-autonomous-vehicles/'
#DATA_PATH = '../input/3d-object-detection-for-autonomous-vehicles/'
DATA_PATH = '../../3d-object-detection-for-autonomous-vehicles/'

## 1 load test data set

In [10]:
test=pd.read_csv(DATA_PATH+'sample_submission.csv')

test_dataset = LyftDataset(data_path='.', json_path='../../3d-object-detection-for-autonomous-vehicles/test_data', verbose=True)

JSON file instance.json missing, using empty list
JSON file sample_annotation.json missing, using empty list
9 category,
17 attribute,
4 visibility,
0 instance,
8 sensor,
168 calibrated_sensor,
219744 ego_pose,
218 log,
218 scene,
27468 sample,
219744 sample_data,
0 sample_annotation,
1 map,
Done loading in 1.8 seconds.
Reverse indexing ...
Done reverse indexing in 0.8 seconds.


## function global_from_rect(Output transformation)

In [11]:
def label_global_from_rect(sample_ID,args):
    #args = [x,y,z,width,length,height,yaw]
     # get the label from the sample_ID
    import math
    sample_token=sample_ID 
    sample = test_dataset.get("sample", sample_token)

    sample_lidar_token = sample["data"]["LIDAR_TOP"]
    lidar_data = test_dataset.get("sample_data", sample_lidar_token)

    # get lidar calibration info
    ego_pose = test_dataset.get("ego_pose", lidar_data["ego_pose_token"])
    calibrated_sensor = test_dataset.get("calibrated_sensor", lidar_data["calibrated_sensor_token"])

    # get the transformation matrix from calibration info
    Tr_rect_sensor = np.array([[-1,0,0,0],[0,0,1,0],[0,-1,0,0],[0,0,0,1]])

    global_from_car = transform_matrix(ego_pose['translation'],
                                       Quaternion(ego_pose['rotation']), inverse=False)
    car_from_sensor = transform_matrix(calibrated_sensor['translation'], Quaternion(calibrated_sensor['rotation']),
                                        inverse=False)
    global_from_sensor = np.dot(global_from_car,car_from_sensor)
    
    global_from_rect = np.dot(global_from_sensor,Tr_rect_sensor)
    
    center_x = args[0]
    center_y = args[1]
    center_z = args[2]
    width = args[3]
    length = args[4]
    height = args[5]
    yaw = args[6]
        
        
        
    # transfrom the center point from global to sensor (center_x,center_y,center_z)->(x,y,z)
    tmp = np.array([[center_x],[center_y],[center_z],[1.0]])
    coor = np.dot(global_from_rect,tmp)
    x = coor[0][0]
    y = coor[1][0]
    z = coor[2][0]

    # transform the yaw from sensor to global yaw->yaw_sensor
    # yaw vector += center vector
    x_yaw = center_x - np.sin(yaw)
    z_yaw = center_z + np.cos(yaw)
    y_yaw = center_y
    # transfer yaw vector to sensor
    coor_yaw = np.dot(global_from_rect,np.array([[x_yaw],[y_yaw],[z_yaw],[1.0]]))
    x_yaw_global = coor_yaw[0][0]
    y_yaw_global = coor_yaw[1][0]
    z_yaw_global = coor_yaw[2][0]
    # the angle between yaw_sensor and the y axis
    #yaw_new = np.arccos(np.clip(np.dot((1,0),(y_yaw_global - y,x_yaw_global - x)), -1.0, 1.0))
    yaw_new = math.atan2(-(x_yaw_global - x),y_yaw_global - y)
    
        
    return [x,y,z,width,length,height,yaw_new]


## lidar filepath_from id

In [46]:

def lidar_filepath_from_id(sample_id):
    sample_token=ID
    sample = test_dataset.get("sample", sample_token)

    sample_lidar_token = sample["data"]["LIDAR_TOP"]
    
    lidar_path = test_dataset.get_sample_data_path(sample_lidar_token)
    return lidar_path

    

In [47]:
lidar_filepath
for ID in test.Id[0:5]:
    
    print(lidar_filepath_from_id(ID))

/home/lingling/Downloads/Lyft_PointRCNN/tools/lidar/host-a011_lidar1_1233961845901454486.bin
/home/lingling/Downloads/Lyft_PointRCNN/tools/lidar/host-a004_lidar1_1233427459701683166.bin
/home/lingling/Downloads/Lyft_PointRCNN/tools/lidar/host-a007_lidar1_1233683925701293606.bin
/home/lingling/Downloads/Lyft_PointRCNN/tools/lidar/host-a004_lidar1_1233521280601485046.bin
/home/lingling/Downloads/Lyft_PointRCNN/tools/lidar/host-a007_lidar1_1233007424700531806.bin


## calib file

In [26]:
import os
def write_calib(calib_folder,idx):
    
    label_save_fn = os.path.join(calib_folder,"{:06n}".format(idx)+'.txt')
    Tr_lyft_rect = np.array([-1,  0,  0,  0, 0,  0, -1,  0, 0,  1,  0,  0])
    R0_rect = np.array([1,0,0,0,1,0,0,0,1])
    with open(label_save_fn, 'w') as f:

        print('P0: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f'  %
                          (1,0,0,0,0,1,0,0,0,0,1,0),file=f)
        print('P1: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f'  %
                          (1,0,0,0,0,1,0,0,0,0,1,0),file=f)
        print('P2: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f'  %
                          (1,0,0,0,0,1,0,0,0,0,1,0),file=f)
        print('P3: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f'  %
                          (1,0,0,0,0,1,0,0,0,0,1,0),file=f)

        print('R0_rect: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f' %
                          (R0_rect[0], R0_rect[1],R0_rect[2], R0_rect[3], R0_rect[4],
                           R0_rect[5], R0_rect[6], R0_rect[7], R0_rect[8]),file=f)

        print('Tr_velo_to_cam: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f' %
                          (-1,0,0,0,0,0,-1,0,0,1,0,0),file=f)

In [38]:
test.Id

0        7eb3e546df5311b035f1d4b7e88351ffdd85f311ef1bdd...
1        a76d702996a086ecd4add92b6c533b76f0ab1dd35a4729...
2        a9919963e5cd83d12b60bb9c386869ecb4dffcf2e3c401...
3        e3f42dc16cb366723699a5e12098edb56907d676cb5720...
4        033edef20c7f951738c037c0275dc02279a15454fda3e6...
                               ...                        
27463    53f4dfacb709a170445e4ad7b5a0688728c77ff3cd0fe5...
27464    854794e664c180c96527feba0fa8dab4ae491c5b784fc6...
27465    83ed8e28c6ababc1607e7c04262221e3fa9d41846185a3...
27466    f18acdfc03c6f092864487063b31c1914d631d992295ef...
27467    aab521819add7f1a683de63702a8891e6f9e9c35eec497...
Name: Id, Length: 27468, dtype: object

## main


In [40]:
import os
calib_folder = "/home/lingling/Downloads/Lyft_PointRCNN/data/KITTI/object/testing/calib/"
for i,sample_id in enumerate(test.Id):
    
    write_calib(calib_folder,i)
    print(sample_id)
    lidar_filepath = lidar_filepath_from_id(sample_id)
    print(lidar_filepath)
    if i >200:
        break;
    # move lidar to path
    lidar_save_name = "{:06n}".format(i)+'.bin'
    lidar_save_fn = '/home/lingling/Downloads/Lyft_PointRCNN/data/KITTI/object/testing/'+lidar_save_name
    copyfile(lidar_filepath,lidar_save_fn) 
    # predition - arges
    
    #label = label_global_from_rect(sample_ID,args)
    
    # remove the lidar_file
    
    os.remove(lidar_save_fn)
    # write label to csv file
    
    
    
    
    

7eb3e546df5311b035f1d4b7e88351ffdd85f311ef1bdd22f71a160e92386fed
/home/lingling/Downloads/Lyft_PointRCNN/tools/lidar/host-a007_lidar1_1233007424700531806.bin
a76d702996a086ecd4add92b6c533b76f0ab1dd35a47295a618c16cbfd529b6e
/home/lingling/Downloads/Lyft_PointRCNN/tools/lidar/host-a007_lidar1_1233007424700531806.bin
a9919963e5cd83d12b60bb9c386869ecb4dffcf2e3c401829965ea04a340b3b4
/home/lingling/Downloads/Lyft_PointRCNN/tools/lidar/host-a007_lidar1_1233007424700531806.bin
e3f42dc16cb366723699a5e12098edb56907d676cb5720cde10f1d6d3a41d6e1
/home/lingling/Downloads/Lyft_PointRCNN/tools/lidar/host-a007_lidar1_1233007424700531806.bin
033edef20c7f951738c037c0275dc02279a15454fda3e6e269d215ad2acc66c7
/home/lingling/Downloads/Lyft_PointRCNN/tools/lidar/host-a007_lidar1_1233007424700531806.bin
c386141fc79587ed49ba0a9acd59c10d3d2c10e7b99479fd156da084a7082a15
/home/lingling/Downloads/Lyft_PointRCNN/tools/lidar/host-a007_lidar1_1233007424700531806.bin
c7b0095cd5470e9a4624ebcc83fca08654c4bd4996198b1614ec

7de8dcc56cdade8e5335b545b097fff1e0418fb5f8d7c921491d39f27802508c
/home/lingling/Downloads/Lyft_PointRCNN/tools/lidar/host-a007_lidar1_1233007424700531806.bin
47222357a41f818c874d82b9a77ac44e538ae444fca55d924c2fc2bc7dcebacb
/home/lingling/Downloads/Lyft_PointRCNN/tools/lidar/host-a007_lidar1_1233007424700531806.bin
3e364a6f347cb17a0ed1a5800c70a348b5de2a24f0e5223c13d736ee18889557
/home/lingling/Downloads/Lyft_PointRCNN/tools/lidar/host-a007_lidar1_1233007424700531806.bin
011a698fc69bf2566a9bb58e2c44d1bb46a1609860f564ea37d754035d002cdf
/home/lingling/Downloads/Lyft_PointRCNN/tools/lidar/host-a007_lidar1_1233007424700531806.bin
2162a3da83762e6b89df27ba0f18b0e46a0e175927507e968c79162e9c21f855
/home/lingling/Downloads/Lyft_PointRCNN/tools/lidar/host-a007_lidar1_1233007424700531806.bin
4d581676cfefadaff428ea69d9948da155c601b7add16267721fb0b7586d4e77
/home/lingling/Downloads/Lyft_PointRCNN/tools/lidar/host-a007_lidar1_1233007424700531806.bin
94853225586bccda4ed456e2141954d0ea62867a30fb6f1b4f9d

In [22]:
!pwd

/home/lingling/Downloads/Lyft_PointRCNN/tools
